In [1]:
/* 30-5 */
USE master;
GO
CREATE LOGIN TestUser WITH PASSWORD = 'abcdelllllll!'
USE AdventureWorks2014;
GO
CREATE USER TestUser FROM LOGIN TestUser;
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.013

In [2]:
/* Next, I’ll grant and deny various permissions. */

USE AdventureWorks2014;
GO
GRANT SELECT ON HumanResources.Department TO TestUser;
DENY SELECT ON Production.ProductPhoto TO TestUser;
GRANT EXEC ON HumanResources.uspUpdateEmployeeHireInfo TO TestUser;
GRANT CREATE ASSEMBLY TO TestUser;
GRANT SELECT ON Schema::Person TO TestUser;
DENY IMPERSONATE ON USER::dbo TO TestUser;
DENY SELECT ON HumanResources.Employee(BirthDate) TO TestUser;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.021

In [3]:
/* check principals */
USE AdventureWorks2014;
GO
SELECT principal_id
FROM sys.database_principals
WHERE name = 'TestUser';
GO

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.029

principal_id
5


In [4]:
USE AdventureWorks2014;
GO
SELECT
    p.class_desc,
    p.permission_name,
    p.state_desc,
    ISNULL(o.type_desc,'') type_desc,
    CASE p.class_desc
    WHEN 'SCHEMA'
    THEN schema_name(major_id)
    WHEN 'OBJECT_OR_COLUMN'
    THEN CASE
        WHEN minor_id = 0
        THEN object_name(major_id)
        ELSE (SELECT
        object_name(object_id) + '.' + name
        FROM sys.columns
        WHERE object_id = p.major_id 
        AND column_id = p.minor_id) END
    ELSE '' END AS object_name
FROM sys.database_permissions p
LEFT OUTER JOIN sys.objects o 
    ON o.object_id = p.major_id
WHERE grantee_principal_id = 7;
GO

Commands completed successfully.

(0 rows affected)

class_desc,permission_name,state_desc,type_desc,object_name


Total execution time: 00:00:00.081

In [5]:
/* less manual version */
USE AdventureWorks2014;
GO
SELECT
    p.class_desc,
    p.permission_name,
    p.state_desc,
    ISNULL(o.type_desc,'') type_desc,
    CASE p.class_desc
    WHEN 'SCHEMA'
    THEN schema_name(major_id)
    WHEN 'OBJECT_OR_COLUMN'
    THEN CASE
        WHEN minor_id = 0
        THEN object_name(major_id)
        ELSE (SELECT
        object_name(object_id) + '.' + name
        FROM sys.columns
        WHERE object_id = p.major_id 
        AND column_id = p.minor_id) END
    ELSE '' END AS object_name
FROM sys.database_permissions p
LEFT OUTER JOIN sys.objects o 
    ON o.object_id = p.major_id
INNER JOIN sys.database_principals dp
	ON dp.principal_id = p.grantee_principal_id
WHERE dp.name = 'TestUser';
GO

Commands completed successfully.

(8 rows affected)

Total execution time: 00:00:00.118

class_desc,permission_name,state_desc,type_desc,object_name
DATABASE,CONNECT,GRANT,,
DATABASE,CREATE ASSEMBLY,GRANT,,
OBJECT_OR_COLUMN,EXECUTE,GRANT,SQL_STORED_PROCEDURE,uspUpdateEmployeeHireInfo
OBJECT_OR_COLUMN,SELECT,DENY,USER_TABLE,ProductPhoto
OBJECT_OR_COLUMN,SELECT,GRANT,USER_TABLE,Department
OBJECT_OR_COLUMN,SELECT,DENY,USER_TABLE,Employee.BirthDate
SCHEMA,SELECT,GRANT,SYSTEM_TABLE,Person
DATABASE_PRINCIPAL,IMPERSONATE,DENY,,
